In [2]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
from shapely.geometry import *
from shapely.ops import nearest_points
from utils import *
import math

In [3]:
crsEuc,crsDeg=getUsualCRS()
bboxEuc,bboxDeg=getUsualBbox()

Lisbon=bbox(38.8076, 38.6952, -9.0929, -9.2694)
Lisbon1=Lisbon.convertToXY()
neighborsNumber=2
squares=readGeodatafromFile("../datasets/acessibility/squares1.shp",crs=crsEuc)
categories=list(taxonomy().keys())
pois,clusters=loadClusters(categories=categories,crs=crsEuc,folderDir="../datasets/pois/clusters/")

(4694149.275033623, 4678105.154380481, -1031864.8879591699, -1012216.9978341572)


In [4]:
def score1(pointGeom,busDist,distances,nPoints):
    score=0
    for cat in categories:
        dist1=distances[cat][0]
        dist2=distances[cat][1]
        n1=nPoints[cat][0]
        n2=nPoints[cat][1]

        scoreCat=dist1*0.8+dist2*0.2
        score+=scoreCat
    return score

def score2(pointGeom,busDist,distances,nPoints):
    score=0
    for cat in categories:
        dist1=distances[cat][0]
        dist2=distances[cat][1]
        n1=nPoints[cat][0]
        n2=nPoints[cat][1]

        scoreCat=dist1/n1*0.8+dist2/n2*0.2
        score+=scoreCat
    return score

def score3(pointGeom,busDist,distances,nPoints):
    score=0
    for cat in categories:
        dist1=distances[cat][0]
        dist2=distances[cat][1]
        n1=nPoints[cat][0]
        n2=nPoints[cat][1]

        scoreCat=busDist*2+dist1/n1*0.8+dist2/n2*0.2
        score+=scoreCat
    return score

def score4(pointGeom,busDist,distances,nPoints):
    score=0
    i=0
    for cat in categories:
        dist1=distances[cat][0]
        dist2=distances[cat][1]
        n1=nPoints[cat][0]
        n2=nPoints[cat][1]

        scoreCatAll=busDist*2+dist1/n1*0.8+dist2/n2*0.2
        scoreCatBus=busDist*2
        ratio=scoreCatAll/scoreCatBus
        score+=ratio
        i+=1
    return ratio/i
    

In [5]:
scoreDF=squares.copy()
for i in tqdm(range(len(squares))):
    geom=squares.loc[i,'geometry'].centroid
    busDist=squares.loc[i,'stopDist']
    dists={}
    nPoints={}
    for cat in categories:
        clustCat=clusters[cat]
        poisCat=pois[cat]
        aux=[]
        aux1=[]
        for k in range(neighborsNumber):
            name="{}{}ID".format(cat[0:4],k)
            name1="{}{}Dist".format(cat[0:5],k)
            clusterID=squares.loc[i,name]
            dist=squares.loc[i,name1]
            n=getCellWithValue(geoDataFrame=clustCat,columnOfValue='clusterID',columnToSelect='nPoints',value=clusterID)

            dists[cat]=dists.get(cat,[])+[dist]
            nPoints[cat]=nPoints.get(cat,[])+[n]

        nPoints[cat] = [x for _,x in sorted(zip(dists[cat],nPoints[cat]))]
        dists[cat].sort()

    score=score4(busDist=busDist,distances=dists,nPoints=nPoints,pointGeom=geom)
    scoreDF.at[i,'score']=score

writeGeodataToGis(geodf=scoreDF,targetFile='../datasets/acessibility/score4.shp',crs=crsEuc)


  0%|          | 0/31717 [00:00<?, ?it/s]C:\Users\franc\AppData\Local\Temp\ipykernel_26948\3555747832.py:48: RuntimeWarning: divide by zero encountered in double_scalars
  ratio=scoreCatAll/scoreCatBus
100%|██████████| 31717/31717 [03:58<00:00, 133.06it/s]
c:\Users\franc\anaconda3\envs\gis\lib\site-packages\geopandas\io\file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
